# HOMEWORK FOR QSTFIN
#### Author: Ivan Cherepakhin
#### email: icherepaxin@bk.ru
#### tg: @User1Usr
Дата получения задания: 04.11 \
Дата сдачи задачния: 07.11

## 1. MAKE STATEMENT

Надо посторить файл problem_test_labels.csv, в котором для каждого объекта из test написана вероятность принадлежности к категории labels. Для проверки качества результат используем метрику LogLoss(по каждой категории считаем отдельно и потом берется среднее от них).

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

Рассмотрим количество пропусков в labels.

In [ ]:
df_labels = pd.read_csv("problem_labels.csv")
df_labels.info()

Из условия и этих меток ясно, что задача является multilabel classification. Для ее решения есть множество различных моделей, которые мы проверим далее.

In [ ]:
df_train = pd.read_csv("problem_train.csv")
df_train

In [ ]:
df_test = pd.read_csv("problem_test.csv")
df_test

## 2. DATA TRANSFORM AND ANALYSIS

### 2.1. Work with NaN

Поработаем с пропусками.

In [ ]:
df_train.replace('NaN', np.nan, inplace=True) # замена строк NaN на настоящие
print("Полностью заполненных столбцов = ", df_train.notna().all().sum())

In [ ]:
print("Есть пустые слобцы = ", df_train.shape[1] != df_train.isnull().all().sum())

Убирем эти столбцы из train и test, тк они не помогут в обучении.

In [ ]:
names_col_filtr = df_train.isnull().all()
all_nan_col = names_col_filtr.index * names_col_filtr.values
all_nan_col = all_nan_col.drop_duplicates()[1:]
df_train = df_train.drop(columns=all_nan_col)
df_test = df_test.drop(columns=all_nan_col)

Поработаем сначала с полностью очищенной от NaN датой.

In [ ]:
full_nonNan_df_train = df_train.dropna(axis=1)


Заметим, что в test те же столбцы, кроме одного, являются полностью заполнеными. Это говорит нам о некоторой закономерности заполнения данных.

In [ ]:
full_nonNan_df_test = df_test[full_nonNan_df_train.columns]
check_to_nan = full_nonNan_df_test.notna().all()
print("Столбцов, которые полностью заполнены = {}(из {})".format(full_nonNan_df_test.notna().all().sum(), full_nonNan_df_test.shape[1]))
col_of_nan = check_to_nan[check_to_nan == False].index
col_of_nan


Заметим, что отсутсвующее значение только в одной строке. Возьмем id этой стороки(может пригодиться для дальнейшего изучения).

In [ ]:
index_of_nan = full_nonNan_df_test['c_1348'][full_nonNan_df_test['c_1348'].isna() == True].index
full_nonNan_df_test = full_nonNan_df_test.dropna()
index_of_nan

### 2.2. Work with dublicates

Теперь проверим на наличие дубликатов id в test и train.

In [ ]:
set(full_nonNan_df_train.id) & set(full_nonNan_df_test.id) == set()

Проверка на совпадение типов в test и train.

In [ ]:
sum(full_nonNan_df_train.dtypes == full_nonNan_df_test.dtypes) == len(full_nonNan_df_train.dtypes)

### 2.3. Work with category

Разберемся, что делать с нечисловыми переменными. Сложно понять какие фичи являются просто переменными, а какие конкретными значениями, для каких существенным является порядок, а для каких нет.

In [ ]:
print("Нечисловых столбцов = ", sum(full_nonNan_df_train.dtypes == "object"))

На первый взгяд кажется, что в числовых фичах много дубликатов. Избавимся от них, дабы избежать overfit.

In [ ]:
full_nonNan_df_train = full_nonNan_df_train.T.drop_duplicates().T

In [ ]:
full_nonNan_df_test = full_nonNan_df_test[full_nonNan_df_train.columns]

Рассмотрим столбец release. Можно предположить, что этот слолбец все таки является категориальным, тк мы нам дано название столбца и в нем всего 3 состояния. Применим one-hot encoder для преобразования в числовой вариант.

In [ ]:
full_nonNan_df_train = pd.get_dummies(full_nonNan_df_train, columns=["release"])
full_nonNan_df_test = pd.get_dummies(full_nonNan_df_test, columns=["release"])
full_nonNan_df_train = full_nonNan_df_train.apply(pd.to_numeric, errors='ignore')
full_nonNan_df_test = full_nonNan_df_test.apply(pd.to_numeric, errors='ignore')

Рассмотрим количество уникальных значений в категориальных столбцах, чтобы оценить, насколько увеличится датасет при one-hot и чтобы оптимально и без потерь преобразовать категории.

In [ ]:
list_of_category_variable = full_nonNan_df_train.dtypes.loc[full_nonNan_df_train.dtypes == "object"].index
list_of_num_variable = full_nonNan_df_train.dtypes.loc[full_nonNan_df_train.dtypes != "object"].index
unique_elem_in_col = {}

for col in list_of_category_variable:
    unique_elem_in_col[col] = list(zip(full_nonNan_df_train[col].value_counts().to_dict().keys(), full_nonNan_df_train[col].value_counts().to_dict().values()))

unique_elem_in_col

In [ ]:
list_of_category_variable = full_nonNan_df_test.dtypes.loc[full_nonNan_df_test.dtypes == "object"].index
list_of_num_variable = full_nonNan_df_test.dtypes.loc[full_nonNan_df_test.dtypes != "object"].index
unique_elem_in_col = {}

for col in list_of_category_variable:
    unique_elem_in_col[col] = list(zip(full_nonNan_df_test[col].value_counts().to_dict().keys(), full_nonNan_df_test[col].value_counts().to_dict().values()))

unique_elem_in_col

Заметим, что c_1158 и c_1348 в train содержат такие категории у которых маленькая частота. К сожалению, какого то четкого порядка и зависимостей пока не удается заметить.

In [ ]:
onehotencoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")

In [ ]:
data_new = onehotencoder.fit_transform(full_nonNan_df_train[list_of_category_variable])
full_nonNan_df_train = pd.concat([full_nonNan_df_train,data_new], axis=1).drop(columns=list_of_category_variable)

In [ ]:
data_new = onehotencoder.transform(full_nonNan_df_test[list_of_category_variable])
full_nonNan_df_test = pd.concat([full_nonNan_df_test, data_new], axis=1).drop(columns=list_of_category_variable)

### 2.4 Ploting

Отрисуем числовые фичи.

In [ ]:
full_nonNan_df_train[list_of_num_variable].drop(columns=["id", "release_a", "release_b", "release_c", "n_0047"]).hist(figsize=(18,12), bins=40)

Говорить о каком то распределние трудно, но можно заметить, что n_0078 и n_0108 очень похожи.

## 3. DATA SCIENCE

Попробуем применять методы машинного обучения. Вначале масштабируем данные.

In [ ]:
scaler = StandardScaler().set_output(transform="pandas")

Не будем масштабировать int фичи, так как могут быть категориями.

In [ ]:
X = full_nonNan_df_train.copy()
X.set_index("id", inplace=True)

In [ ]:
main_test = full_nonNan_df_test.copy()
main_test.set_index("id", inplace=True)

In [ ]:
y = pd.read_csv("problem_labels.csv").set_index('id')
y_col = y.columns
y

Для начальной проверки моделей, разобьем train на обчающую и тестовую часть.

In [ ]:
y.columns = np.arange(0, len(y.columns))
X = X.to_numpy()
y = y.to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Добавим следующие трансофрмеры для pipline.

In [ ]:
from numba import njit

In [ ]:
slc = StandardScaler()

In [ ]:
pca = PCA()

In [ ]:
def custom_metrics_log_loss_by_col(y_true, y_pred):
    y_pred = np.array(y_pred)
    y_true = y_true.T
    mean_log_loss = 0

    # надо оптимизировать
    # написать свою векторизированную функцию log_loss и применить njit из numba
    for i in range(y_true.shape[1]):
        mean_log_loss += log_loss(y_true[:,i], y_pred[:,i,:])

    return mean_log_loss / y_true.shape[1]

custom_log_loss = make_scorer(custom_metrics_log_loss_by_col, needs_proba=True)

Настроим модель DecisionTreeClassifier.

In [35]:
clf = MultiOutputClassifier(DecisionTreeClassifier(random_state=0))

pipe_tree = Pipeline(
            steps=[
                    ('pca', pca),
                    ('tree', clf)
])


n_components = [5, 10]

param = [{
            'pca__n_components': n_components,
            'tree__estimator__max_depth': [6, 12],
            'tree__estimator__max_features': [5, 10]
}]

dtc_model = GridSearchCV(pipe_tree, param, scoring=custom_log_loss, cv=5).fit(X_train, y_train)

In [36]:
tree_best_estim = dtc_model.best_estimator_
y_test_pred = tree_best_estim.predict_proba(X_test)
y_test_score = custom_metrics_log_loss_by_col(y_test, y_test_pred)
y_test_score

3.24821543254688

Настроим модель kNN.

In [ ]:
clf = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=1))

pipe_knn = Pipeline(
            steps=[ ('slc', slc,),
                    ('pca', pca),
                    ('kNN', clf)
])


param = [{
            'pca__n_components': n_components,
            'kNN__estimator__n_neighbors': np.arange(1, 30, 5),
}]

knn_model = GridSearchCV(pipe_knn, param, scoring=custom_log_loss, cv=5).fit(X_train, y_train)

In [ ]:
knn_best_estim = knn_model.best_estimator_
y_test_pred = knn_best_estim.predict_proba(X_test)
y_test_score = custom_metrics_log_loss_by_col(y_test, y_test_pred)
y_test_score

Настроим MLPClassifier.

In [ ]:
clf = MultiOutputClassifier(MLPClassifier(solver='adam',
                                            learning_rate_init=0.01,
                                            max_iter=300,
                                            activation='relu',
                                            early_stopping=True))

pipe_mlp = Pipeline(
            steps=[ ('slc', slc,),
                    ('pca', pca),
                    ('mlp', clf)
])

param = [{
            'pca__n_components': n_components,
}]

mpl_model = GridSearchCV(pipe_mlp, param, scoring=custom_log_loss, cv=5).fit(X_train, y_train)


In [ ]:
mlp_best_estim = mpl_model.best_estimator_
y_test_pred = mlp_best_estim.predict_proba(X_test)
y_test_score = custom_metrics_log_loss_by_col(y_test, y_test_pred)
y_test_score

Из показателей score на тесте, делаем вывод, что лучше всего обучаться на последней моделе.

In [ ]:
clf = MultiOutputClassifier(MLPClassifier(solver='adam',
                                            learning_rate_init=0.01,
                                            max_iter=300,
                                            activation='relu',
                                            early_stopping=True))

pipe_mlp = Pipeline(
            steps=[ ('slc', slc,),
                    ('pca', pca),
                    ('mlp', clf)
])

n_components = [25, 50, 100]

param = [{
            'pca__n_components': n_components,
            'mlp__estimator__alpha': 10.0 ** -np.arange(1, 5),
}]

mpl_model = GridSearchCV(pipe_mlp, param, scoring=custom_log_loss, cv=5).fit(X, y)

In [ ]:
mpl_best = mpl_model.best_estimator_
main_test_pred = np.array(mpl_best.predict_proba(main_test))
main_test_pred = pd.DataFrame(main_test_pred[:,:,1]).T
main_test_pred.index = main_test.index
main_test_pred.columns = y_col
main_test_pred.to_csv("problem_test_labels.csv", sep=';', encoding='utf-8')